---
##**¡Intenta escapar del laberinto!**
---
Autor: Juan Valverde Santiago

# Bienvenido a este puzzle.

El objetivo de este juego es que puedas programar un agente capaz de encontrar la salida de un laberinto cualquiera. Como detalle importante, la entrada y la salida del laberinto se encontrarán siempre pegados a un borde del laberinto.

Para realizar esta tarea, sigue los pasos que se indican en cada sección.

Pero primero, deberás guardarte una copia de este cuaderno, pues no puedes guardar cambios en el original.

# Descarga e importación

El siguiente bloque obtiene los archivos necesarios para la correcta ejecución del cuaderno.

Simplemente, ejecuta el bloque sin modificar nada.

In [1]:
%%capture
!mkdir v002
!mkdir images
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Agent.py' -O v002/Agent.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment.py' -O v002/Enviroment.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/__init__.py' -O v002/__init__.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment_with_agents.py' -O v002/Enviroment_with_agents.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/InOut_Simple_Laberinth.py' -O v002/InOut_Simple_Laberinth.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelNoTomato.bmp' -O images/PixelNoTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelTomato.bmp' -O images/PixelTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/face1_borders.bmp' -O images/face1_borders.bmp
!wget 'https://raw.githubusercontent.com/juanvs00/AgentSurvival/main/rules_to_clips_v2.py' -O rules_to_clips_v2.py

!pip install clipspy

from v002 import *
from rules_to_clips_v2 import *

# Escritura de las reglas

En este bloque debes trabajar. Aquí es donde debes escribir las reglas CLIPS y el nombre del agente.

Tu objetivo será crear un comportamiento para que el agente escape del laberinto. No puedes modificar el nombre del vector, aunque sí puedes crear un mayor número de reglas.

Las percepciones del agente son los muros que le rodean y los objetos que encuentra, las opciones que puedes usar siguen la siguiente forma:
*   (walls \$? front \$?)
*   (not (walls \$? back \$?))
*   (available_object exit ?x)

Siendo los posibles muros front, back, left y right, que corresponden con los muros de delante, detrás, izquierda y derecha, respectivamente.

Y siendo los posibles objetos que encuentra el agente los siguientes:
*   exit
*   food type 1
*   agent

Por otro lado, las acciones que puede realizar el agente son las siguientes:
*   "self.move_forward()"
*   "self.turn_left()"
*   "self.turn_right()"
*   Funciones de objeto

Estas acciones deben añadirse en el hecho "pending_function_calls", como puedes ver en las reglas de ejemplo. En la causa de la regla se introduce el hecho en una variable; mientras que, en el consecuente se elimina el hecho y se vuelve a crear añadiendo las acciones que quieres que realice tu agente. En cuanto a las funciones de objeto, puedes observar la regla 4 de ejemplo llamada "salir" en la que se realiza la función de salida del laberinto.

Ten en cuenta que las acciones del agente consumen movimientos, y el número de movimientos está limitado.


¡Suerte!

In [2]:
# Creación del entorno CLIPS, no modificar
env = clips.Environment()

# TODO: Aquí debes escribir las reglas CLIPS
rules = []
rule1 = """
    (defrule giraDcha
    (not (walls $? right $?))
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_right()" "self.move_forward()"))
    )"""
rules.append(rule1)
rule2 = """
    (defrule haciaDelante
    (walls $? right $?)
    (not (walls $? front $?))
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )"""
rules.append(rule2)
rule3 = """
    (defrule giraIzqda
    (walls $? right $?)
    (walls $? front $?)
    ?h<-(pending_function_calls $?calls)
    (not (available_object exit ?x))
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_left()"))
    )"""
rules.append(rule3)
rule4 = """
  (defrule salir
  (available_object exit ?x)
  ?h<-(pending_function_calls $?calls)
  =>
  (retract ?h)
  (assert (pending_function_calls ?x))
  )
  """
rules.append(rule4)

# Se definen las reglas en el entorno CLIPS, no modificar
for i in rules:
  env.build(i)
env.reset()

# TODO: Aquí debes escribir el nombre de tu agente
agent_name = "YO!"

# Función move

El siguiente bloque es la función move del agente. Simplemente, ejecuta el bloque de código.

**¡NO DEBES MODIFICARLO!**

In [3]:
def move(self):
  # Elimina los hechos que no necesita del entorno CLIPS
  reset_environment(env)

  # Recoge las paredes y objetos que detecta el agente
  walls, objects = what_I_see(self)

  # Crear el hecho de las paredes en el entorno CLIPS
  set_walls(env,walls)

  # Crear las posibles funciones accesibles al agente
  for i in objects:
        for key in i:
            if key.endswith('_function'):
                object_function = "ob_" + str(id(i)) + "_funct"
                available_objects = {object_function: i[key]}
                new_object = '(available_object ' + str(
                    i['type']) + ' "' + "available_objects" + "['" + object_function + "']" + '(self)")'
                env.assert_string(new_object)

  # Ejecutar pasos de agente
  env.run(1)

  # Ejecutar las opciones en pending_function_calls
  for i, fact in enumerate(env.facts()):
    if str(fact).startswith("(pending_function_calls"):
      functions = str(fact)[:-1].split()[1:]
      for j in functions:
        eval(j.replace('"',''))

# Ejecución del puzzle

Ha llegado el momento de comprobar si tu agente es capaz de escapar del laberinto.

En principio, no debes modificar nada. Aunque puedes modificar algún parámetro de la función InOut_Simple_Laberinth, por ejemplo el primer parámetro es el tamaño del escenario.

In [ ]:
lb1 = InOut_Simple_Laberinth(7, plot_run='always', move_protection=False)
lb1.create_agent(agent_name, move)
lb1.run()

# ¿Es realmente bueno el comportamiento de tu agente?

En el siguiente bloque puedes comprobar cuántas veces es tu agente capaz de escapar del laberinto de un total de ejecuciones.

En principio, no debes modificar nada. Aunque puedes puedes modificar el número de ejecucuciones totales (la variable num_runs).

In [ ]:
from tqdm import tqdm
num_runs = 200
num_success =  0

for _ in tqdm(range(num_runs)):
  lb1 = InOut_Simple_Laberinth(7, plot_run='never', move_protection=False)
  lb1.create_agent(agent_name, move)
  winner = lb1.run()

  if winner is not None:
    num_success += 1

print("\n\nEncontraste la salida en", num_success, "ocasiones de", num_runs)